In [1]:
from gensim.models import Word2Vec
from eunjeon import Mecab
tagger = Mecab()

model = Word2Vec.load('word2vec.model')

In [2]:
text_list=open('analysis_token','r',encoding="utf8")

In [3]:
except_words='흠,예,그게,네,아,어,뭐,음,자,이,저,그,때,우리,여러분,번,요,오,앞,뒤,것,여기,저기,이것,저것,이게,저게,이걸,저걸,이번,저번,이런,저런'
excpet_words=except_words.split(',')

In [4]:
# MAIN

from korean_romanizer import *
import jellyfish


def find_error_word(): #오류가 있는지 확인
    error_word=[] #오류단어를 저장
    for i in jamak_nouns:
        if i not in text_list:
            error_word.append(i)
    return error_word

def nearby_error_word(): #오류 단어 앞뒤의 단어 or 앞의 두 단어(오류단어가 가장 마지막에 위치한 경우) 뽑기
    check_nouns=[]#오류단어 앞뒤의 단어 저장
    for i in range(len(error_word)):
        for j in range(len(jamak_nouns)):
            if error_word[i] == jamak_nouns[j]:
                check_nouns_list=[]
                if j == len(jamak_nouns)-1:
                    check_nouns_list.append(jamak_nouns[j-1])
                    check_nouns_list.append(jamak_nouns[j-2])
                    check_nouns_list.append(jamak_nouns[j-3])
                else:
                    check_nouns_list.append(jamak_nouns[j-1])
                    check_nouns_list.append(jamak_nouns[j+1])
                    check_nouns_list.append(jamak_nouns[j-2])
                check_nouns.append(check_nouns_list)    
    return check_nouns


def check_word_list(): #오류 단어 근처의 단어에 대해 연관성 높은 단어의 리스트를 저장
    word_list=[]
    global model_result
    if check_nouns==[]:
        return []

    else:
        for i in range(len(check_nouns)):
            list_result=[]
            for j in check_nouns[i]:
                try:
                    model_result=model.wv.most_similar(j)
                except:
                    pass
                for k in model_result:
                    if k[0] not in except_words:
                        list_result.append(k[0])
            word_list.append(list_result)
   
    return word_list


def romanizing(): #word_list의 한글 발음을 로마자로 변환
    pronounce=[] # word_list의 발음을 저장.
    if word_list == []:
        return []
    else:
        for i in range(len(word_list)):
            pronounce_list=[]
            for j in range(len(word_list[i])):
                a=Romanizer(word_list[i][j])
                pronounce_list.append(a.romanize())
            pronounce.append(pronounce_list)
    return pronounce



def similarity(): #유사도 측정
    probability=[]
    if error_word == []:
        return []
    else:
        for e in range(len(error_word)):
            a=Romanizer(error_word[e]).romanize()
            prob=[]
            prob1=[]
            prob2=[]
            prob3=[]
            for j in range(len(pronounce[e])):
                prob1.append(jellyfish.jaro_winkler_similarity(a, pronounce[e][j]))
                prob2.append(jellyfish.jaro_similarity(a, pronounce[e][j]))
                prob3.append(1-(jellyfish.levenshtein_distance(a,pronounce[e][j]))/10)
                prob.append(prob3[j]+(prob1[j]+prob2[j])/2)
            probability.append(prob)
    return probability



def word_change(): #오류단어를 교체
    global correct_word
    correct_word=[]
    if error_word==[]:
        return jamak
    else:
        change_word=[]
        for i in range(len(probability)):
            err_word_index=probability[i].index(max(probability[i]))
            correct_word.append(word_list[i][err_word_index])
            
        for a in range(len(jamak)):
            for b in range(len(error_word)):
                if jamak[a] == error_word[b]:
                    jamak[a]=correct_word[b]
    return jamak



In [5]:

f=open(r'C:\Users\IBK\fluid_lecture.txt',encoding="utf8") # 자막이 들어올 위치. 자막을 한 줄씩 받을 예정


while True:
    line=f.readline()
    jamak_nn=tagger.nouns(line) #자막에 나오는 의미있는 명사를 찾기
    

    if not line:
        break
    
    jamak_nouns=[]
    for w in jamak_nn:
        if w not in except_words:
            jamak_nouns.append(w)
    
    if tagger.morphs(line)!=[]: #자막문장을 품사별로 끊기
        jamak=tagger.morphs(line)

    #print('자막 = ',jamak)
    
    #print('자막에 나오는 명사 = ',jamak_nouns)

    error_word=find_error_word()
    #print('오류 단어 = ',error_word)
    
    check_nouns=nearby_error_word()
    #print('오류 단어 근처의 단어 = ',check_nouns)
    
    word_list=check_word_list()
    #print('오류 단어에 위치할 확률이 높은 단어 = ',word_list)
    
    pronounce=romanizing()
    #print('오류 단어에 위치할 확률이 높은 단어의 발음 = ',pronounce)
   
    probability=similarity()
    #print('오류단어와의 발음의 유사성 계산 = ',probability)
    
    change_word=word_change()
    #print('변환된 자막 = ',change_word)
    
    
    # 자막으로 전환
    line=list(line)
    space_list=[]
    for index, value in enumerate(line):
        if value==' ':
            space_list.append(index)
    #print('space_list = ',space_list)
    #print('correct word = ',correct_word)
    #print('error word =', error_word)
    change_word=list(''.join(change_word))
    for i in space_list:
        change_word.insert(i,' ')
    final_jamak=''.join(change_word)
    #print('최종 자막 = ',final_jamak,'\n')
    
        
    #--------------------------------------------------------------음, 흠, 어 이런거 없애는게 나을지 두는게 나을지
    # 단어의 글자수가 다른 경우 띄어쓰기 어떻게 조건을 바꾸어야하지?
    
f.close()
